In [ ]:
from generate import *
from tree_search import *
from meatbag import *

In [ ]:
sl_large_policy = load_model('large_policy_model')
board = np.zeros((15, 15), dtype='float32')
last_action = (0, 0)
turn = 0

time = int(input("Enter algorithm processing time (in seconds):"))
players = [Meatbag(),
           MonteCarloTreeSearch(sl_large_policy, None, time=time, verbose=1)]
if input("Choose your side (black/white):") == "white":
    players.reverse()
while get_rollout_result(board, last_action) is None:
    show_board(board)
    last_action = players[turn % 2].make_move()
    players[(turn + 1) % 2].pass_move(last_action)
    board[last_action] = -1.0 + 2 * (turn % 2)
    turn += 1
show_board(board)
res = get_rollout_result(board, last_action)
if res is None:
    print("Not finished")
elif round(res) == 1:
    print('White won!')
elif round(res) == -1:
    print('Black won!')
else:
    print('Draw!')